In [ ]:
!pip install graphframes

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext
import os
from graphframes import *
import pickle
from pyspark.sql.functions import col,explode,size,first
from pyspark.sql.functions import udf, collect_list
from pyspark.sql.functions import count,collect_list,flatten
from pyspark.sql.functions import desc

In [ ]:
conf = SparkConf()

#url par défaut d'une api kubernetes accédé depuis l'intérieur du cluster (ici le notebook tourne lui même dans kubernetes)
conf.setMaster("k8s://https://kubernetes.default.svc:443")

#image des executors spark: pour des raisons de simplicité on réutilise l'image du notebook
conf.set("spark.kubernetes.container.image", os.environ['IMAGE_NAME'])

# Nom du compte de service pour contacter l'api kubernetes : attention le package du datalab crée lui même cette variable d'enviromment.
# Dans un pod du cluster kubernetes il faut lire le fichier /var/run/secrets/kubernetes.io/serviceaccount/token
# Néanmoins ce paramètre est inutile car le contexte kubernetes local de ce notebook est préconfiguré
# conf.set("spark.kubernetes.authenticate.driver.serviceAccountName", os.environ['KUBERNETES_SERVICE_ACCOUNT']) 

# Nom du namespace kubernetes
conf.set("spark.kubernetes.namespace", os.environ['KUBERNETES_NAMESPACE'])

# Nombre d'executeur spark, il se lancera autant de pods kubernetes que le nombre indiqué.
conf.set("spark.executor.instances", "10")

# Mémoire alloué à la JVM
# Attention par défaut le pod kubernetes aura une limite supérieur qui dépend d'autres paramètres.
# On manipulera plus bas pour vérifier la limite de mémoire totale d'un executeur
conf.set("spark.executor.memory", "4g")

conf.set("spark.kubernetes.driver.pod.name", os.environ['KUBERNETES_POD_NAME'])

# Paramètres d'enregistrement des logs spark d'application
# Attention ce paramètres nécessitent la création d'un dossier spark-history. Spark ne le fait pas lui même pour des raisons obscurs
# import s3fs
# endpoint = "https://"+os.environ['AWS_S3_ENDPOINT']
# fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': endpoint})
# fs.touch('s3://tm8enk/spark-history/.keep')
# sparkconf.set("spark.eventLog.enabled","true")
# sparkconf.set("spark.eventLog.dir","s3a://tm8enk/spark-history")
#ici pour gérer le dateTimeFormatter dépendant de la verion de java...
conf.set("spark.sql.legacy.timeParserPolicy","LEGACY")
conf.set("spark.default.parallelism",10)
conf.set("spark.sql.shuffle.partitions",10)
conf.set("spark.jars.packages","graphframes:graphframes:0.8.1-spark3.0-s_2.12")

In [ ]:
spark = SparkSession.builder.appName("graph").config(conf = conf).getOrCreate()

In [ ]:
vertices = [(1,"A"), (2,"B"), (3, "C")]
edges = [(1,2,"love"), (2,1,"hate"), (2,3,"follow")]

v = spark.createDataFrame(vertices, ["id", "name"])
e = spark.createDataFrame(edges, ["src", "dst", "action"])

premierGraphe = GraphFrame(v, e)
premierGraphe.edges.show()

In [ ]:
schema = pickle.load(open("/home/onyxia/work/schema.p", "rb" ))
print(schema)

In [ ]:
df = spark.read.format("json")  \
    .schema(schema) \
    .load("s3a://projet-spark-lab/diffusion/tweets/input")

In [ ]:
df.select(col("id").alias("id"),
         col("user.id").alias("user_id"),
         col("user.name").alias("name"),
         col("user.followers_count"),
         col("entities.hashtags.text").alias("hashtags"),
         size("entities.hashtags").alias("has_hashtag"),
         col("text"),
         col("entities.user_mentions")) \
 .filter(col("has_hashtag")>0) \
 .head(1) 

In [ ]:
tweeters=df.select(col("user.id").alias("id"), col("user.name").alias("name"))
users_mentionned=df.select(explode(col("entities.user_mentions"))).select(col("col.id").alias("id"),col("col.screen_name").alias("name")).distinct()
vertices=tweeters.union(users_mentionned)
# on définit un udf qui prend le premier élément
udf_first = udf(lambda x: x[0])
# on groupe l'union des deux dataset par id d'utilisateur et on collect 
#sous la forme d'une liste leurs noms qui peuvent différer entre les mentions et le compte
# on applique l'udf pour ne retenir que le nom en tête
final_vertices=vertices.groupby("id").agg(udf_first(collect_list("name")).alias("name"))
final_vertices.show(10, truncate=False)

In [27]:
edges=df.select(col("user.id").alias("user_id"), \
               explode(col("entities.user_mentions")).alias("mention"),
               col("entities.hashtags.text").alias("hashtags"),
               "id") \
 .groupby(col("user_id").alias("src"), \
          col("mention.id").alias("dst")) \
 .agg(count("id").alias("nb"),
      flatten(collect_list("hashtags")).alias("hashtags"),
      collect_list("id").alias("id"))
edges.show(10)

+-------+------------------+---+--------+--------------------+
|    src|               dst| nb|hashtags|                  id|
+-------+------------------+---+--------+--------------------+
|1012981|973130412628246529|  1|      []|[1385318639994290...|
|1349111|         217473382|  1|      []|[1426382322706919...|
|1536651|          17510568|  1|      []|[1382995514933850...|
|1745171|          16649457|  1|      []|[1436253710137667...|
|3639301|        4041503175|  1|      []|[1386964092682964...|
|4478161|         217473382|  1|      []|[1433405191211130...|
|4478161|        3077516146|  1|      []|[1394763627283066...|
|5523462|         268227446|  1|      []|[1394285638040567...|
|5891532|           5788732|  1|      []|[1394724638886875...|
|6274062|         200659061|  1|      []|[1425104376738336...|
+-------+------------------+---+--------+--------------------+
only showing top 10 rows



In [ ]:
g = GraphFrame(final_vertices, edges)

In [ ]:
g.cache()

In [ ]:
print("le graphe a "+ str(g.vertices.count()) +" noeuds et "+ str(g.edges.count()) + " arcs")

In [ ]:
pagerank = g.pageRank(resetProbability=0.15,tol=0.01)

In [ ]:
pagerank.vertices.sort(desc("pagerank")).show(truncate=False)

In [ ]:
pagerank.edges.where((col("src")=="103918784") ).show(truncate=False)
pagerank.edges.where((col("dst")=="103918784") ).count()

In [ ]:
g.inDegrees.join(g.vertices,"id").orderBy(desc("inDegree")).show(10,False)

In [ ]:
g.outDegrees.join(g.vertices,"id").orderBy(desc("outDegree")).show(10,False)

In [ ]:
spark.stop()